In [2]:
import requests
import bs4 
import pandas as pd

from libs import dict_to_query

In [3]:
base_url = 'https://www.hrd.go.kr/jsp/HRDP/HRDPO00/HRDPOA90/HRDPOA90_1.jsp'

In [4]:
code = {
    '훈련지역 대분류 코드':'00',
    '훈련지역 중분류 코드':'01',
    'KECO 대분류 코드':'02',
    'KECO 중분류 코드':'03',
    'KECO 소분류 코드':'04',
    'NCS 대분류 코드':'05',
    'NCS 중분류 코드':'06',
    'NCS 소분류 코드':'07',
    'NCS 세분류 코드':'08',
    '훈련종류 코드':'09',
    '훈련방법 코드':'10',
    '훈련기관 구분코드':'11',
}

In [5]:
opt = {
    'authKey':'hfI6LTOnqD5609y88FO2DHzf2OGCuFeO',
    'returnType':'XML',
    'srchType':'None',
    'outType':'1'
}

In [6]:
# 훈련지역 중분류 코드를 제외한 나머지 공통코드

for k,v in code.items():
    opt['srchType'] = v
    
    res = requests.get('?'.join([base_url,dict_to_query(opt)]))
    xml = bs4.BeautifulSoup(res.content,'lxml-xml')
    
    c_name = [i.text for i in xml.findAll('rsltName')]
    c_val = [i.text for i in xml.findAll('rsltCode')]
    
    if c_name.__len__() != c_val.__len__():
        continue
    
    file_name = '../files/common_code/'+'_'.join(k.split())+'.csv'
    pd.DataFrame({
        '코드값':c_val,
        '코드명':c_name
        }).to_csv(file_name,index=False)


In [7]:
# 훈련지역 중분류 코드 누락으로 수작업

import json

with open('../files/훈련지역_중분류_코드.json','rb') as f:
    data = json.load(f)
    pd.DataFrame({
        '코드값':data['코드값'],
        '코드명':data['코드명'],
        }).to_csv('../files/common_code/훈련지역_중분류_코드.csv',index=False)